<a href="https://colab.research.google.com/github/UW-CTRL/AA548-spr23/blob/main/homework/hw_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AA/EE/ME 548: Homework 03

## Name:

### Export page instructions:
Export the PDF as though you were going to print the page in your browser (Ctrl+P) rather than through Colab's print functionality, otherwise some of the text will be cut off.

### Homework feedback:

**What was the coolest thing(s) you learned from lectures/homework that you didn't know previously?**



**Which problem did you find the most challenging and why?**



**Approximate how long did you spend on this homework?**


**Feedback on this homework (can improve for future hw and years)**


## Submission details
To submit your homework, please save a PDF copy of your notebook (make sure all the cells are run correctly, and the correct outputs are displayed), and submit via Canvas.

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
from jax.typing import ArrayLike
import matplotlib.pyplot as plt

## Problem 1

### (a)

# Problem 2

See separate notebook

# Problem 3

In [ ]:
!pip install --upgrade hj-reachability

In [ ]:
from IPython.display import HTML
import matplotlib.animation as anim
import plotly.graph_objects as go

import hj_reachability as hj
from hj_reachability import dynamics
from hj_reachability import sets

In [ ]:

class DynUnicycle(dynamics.ControlAndDisturbanceAffineDynamics):

    def __init__(self,
                 max_turn_rate=1.,
                 max_accel=2.,
                 control_mode="max",
                 disturbance_mode="min",
                 control_space=None,
                 disturbance_space=None,
                 alpha=0.):
        # control and disturbance bounds
        # in this set up, the disturbance bounds are proportional to the control
        # bounds by a factor of alpha
        if control_space is None:
            control_space = sets.Box(lo=jnp.array([-max_turn_rate, -max_accel]), hi=jnp.array([max_turn_rate, max_accel]))
        if disturbance_space is None:
            disturbance_space = sets.Box(lo=jnp.array([-max_turn_rate * alpha, -max_accel * alpha]), hi=jnp.array([max_turn_rate * alpha, max_accel * alpha]))
        super().__init__(control_mode, disturbance_mode, control_space, disturbance_space)

    def open_loop_dynamics(self, state, time):
        ### PUT YOUR CODE HERE ###

    def control_jacobian(self, state, time):
        ### PUT YOUR CODE HERE ###

    def disturbance_jacobian(self, state, time):
        ### PUT YOUR CODE HERE ###


In [ ]:

### REACH ###
# control_mode = "min"
# disturbance_mode = "max"
####################################

### AVOID or when computing FRS ###
control_mode = "max"
disturbance_mode = "min"
####################################

# model with noise/disturbance bounds proportional to the control limits
dynamics_noisy = DynUnicycle(alpha=0.6, control_mode=control_mode, disturbance_mode=disturbance_mode)

# no noise on the model
dynamics = DynUnicycle(control_mode=control_mode, disturbance_mode=disturbance_mode)

grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(lo=np.array([-10., -10., 0., -4.]),
                                                                   hi=np.array([10., 10., 2 * np.pi, 4.])), (21, 21, 30, 31),
                                                       periodic_dims=2)
# terminal values. circle in position space centered at origin and with radius 3
values = jnp.linalg.norm(grid.states[..., :2], axis=-1) - 3
solver_settings = hj.SolverSettings.with_accuracy("very_high",
                                                  hamiltonian_postprocessor=hj.solver.backwards_reachable_tube)

In [ ]:
# solve the HJI equation
time = 0.
target_time = -2  
target_values = hj.step(solver_settings, dynamics, grid, time, values, target_time).block_until_ready()
target_values_noisy = hj.step(solver_settings, dynamics_noisy, grid, time, values, target_time).block_until_ready()


In [ ]:
## plotting
fontsize = 16
## When plotting, use the following indices for theta and v when 
## slicing the value function across the x and y dimensions.
theta_i = 0
v_i = 25

plt.jet()
plt.figure(figsize=(8, 6))
plt.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], target_values[:, :, theta_i, v_i].T)
cbar = plt.colorbar()
plt.contour(grid.coordinate_vectors[0],
            grid.coordinate_vectors[1],
            values[:, :, theta_i, v_i].T,
            levels=0,
            colors="black",
            linewidths=3)
plt.contour(grid.coordinate_vectors[0],
            grid.coordinate_vectors[1],
            target_values[:, :, theta_i, v_i].T,
            levels=0,
            colors="red",
            linewidths=3)
plt.contour(grid.coordinate_vectors[0],
            grid.coordinate_vectors[1],
            target_values_noisy[:, :, theta_i, v_i].T,
            levels=0,
            colors="yellow",
            linewidths=3)



plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
ticklabs = cbar.ax.get_yticklabels()
cbar.ax.set_yticklabels(ticklabs, fontsize=fontsize)
plt.axis("equal")
plt.tight_layout() 

# Problem 4

## 4(a) 

[your answer here]

## 4(b)

In [ ]:
import cvxpy as cp


T = 50  # number of time steps
n = 4   # state dimension
m = 2   # control dimension


xs = cp.Variable([T+1,n]) # state trajectory variable
us = cp.Variable([T,m])   # control trajectory variable

### Add your code here

######################


# plotting out results.
states = xs.value 
controls = us.value

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(states[:,0], states[:,1])
plt.quiver(states[:,0], states[:,1], states[:,2], states[:,3], scale=30, width=.003)
plt.quiver(states[:-1,0], states[:-1,1], controls[:,0], controls[:,1], scale=30, width=.003, color='red')
plt.title("Optimal Trajectory with velocity and acceleration vector")
plt.xlabel("$x$ position")
plt.ylabel("$y$ position")
plt.grid()

plt.subplot(1,2,2)
plt.plot(controls)
plt.plot(np.linalg.norm(controls, 2, axis=1), '--')
plt.title("Optimal Control Sequence")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration")
plt.legend(["$a_x$", "$a_y$", "$\|a\|_2$"])
plt.grid()